In [193]:
import pandas as pd

In [194]:
df = pd.read_csv("sales.csv",sep=";")

In [195]:
df.describe()

,mois,annee,commandes,nr_articles,total_vente,facture,rembourse,tva,expedition,vente_escompte,annule
count,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000,66.000000
mean,6.227273,2015.272727,523.060606,774.424242,206657.541818,196177.349545,8502.895000,35322.081212,1475.468333,3411.762273,116373.224394
std,3.467633,1.612885,380.228348,478.317277,119868.598489,119823.113595,30009.215034,20712.389071,1516.808813,2847.747330,78179.050715
min,1.000000,2013.000000,56.000000,121.000000,54241.630000,164.460000,0.000000,8898.190000,128.670000,109.000000,9860.660000
25%,3.000000,2014.000000,230.500000,407.000000,113054.852500,106968.980000,670.810000,18760.632500,431.105000,1338.930000,56072.590000
50%,6.000000,2015.000000,390.000000,724.500000,152321.650000,142603.875000,1899.100000,27267.585000,924.445000,2680.235000,93295.875000
75%,9.000000,2017.000000,729.000000,1055.750000,284781.205000,267630.997500,3892.995000,49219.915000,1903.240000,4229.325000,168157.362500
max,12.000000,2018.000000,1536.000000,2021.000000,501300.160000,501300.160000,232548.050000,86168.040000,7958.560000,14321.400000,337341.680000


In [196]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib

In [197]:
plotly.offline.init_notebook_mode(connected=True)

In [199]:
import numpy as np

dataset = pd.concat([pd.get_dummies(df["mois"]),df[["total_vente","annee"]]],axis=1)

months = [1,2,3,4,5,6,7,8,9,10,11,12]
mean = []
diff = []

for month in months:
    val = dataset[dataset[month] == 1]["total_vente"].mean()
    mean.append(val)
    if (month <= 6):
        diff.append(dataset[(dataset[month] == 1) & (dataset["annee"] == 2018)]["total_vente"].values[0] / val - 1)
    

dataset.groupby("annee").total_vente.sum()


annee
2013    2594333.05
2014    2308471.74
2015    2413067.55
2016    2687874.89
2017    2516430.70
2018    1119219.83
Name: total_vente, dtype: float64

In [200]:
years = [2013,2014,2015,2016,2017,2018]
data = []

for year in years:
    data.append(go.Scatter(x=df[df["annee"] == year]["mois"],y=df[df["annee"] == year]["total_vente"],name="CA "+str(year)))

predictions = []
for i,m in enumerate(mean):
    if i > 5:
        predictions.append(m * (1 + np.array(diff).mean() ))     
data.append(go.Scatter(x=months,y=mean,name="Moyenne CA"))
data.append(go.Scatter(x=[7,8,9,10,11,12],y=predictions,name="Prediction CA"))

plotly.offline.iplot(data)

In [214]:
summary = go.Scatter(x=[2013,2014,2015,2016,2017,2018],y=df[(df["mois"].isin([1,2,3,4,5,6]))].groupby("annee")["total_vente"].sum().values, name="CA 6 premiers mois")
layout = go.Layout(
    title='CA 6 premiers mois / an')
fig = go.Figure(data=[summary], layout=layout)
plotly.offline.iplot(fig)

In [201]:
print("Chiffre d'affaires estimé 2018: {} €".format(int(round(dataset[dataset["annee"] == 2018]["total_vente"].sum() + np.array(predictions).sum()))))

Chiffre d'affaires estimé 2018: 1831860 €
